In [ ]:
import pandas as pd
import numpy as np
import requests
import re
import csv

In [ ]:
pythia_fragment_CP5 = """
from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunesRun3ECM13p6TeV.PythiaCP5Settings_cfi import *
from Configuration.Generator.PSweightsPythia.PythiaPSweightsSettings_cfi import *

generator = cms.EDFilter("Pythia8ConcurrentHadronizerFilter",
                         maxEventsToPrint = cms.untracked.int32(1),
                         pythiaPylistVerbosity = cms.untracked.int32(1),

                         pythiaHepMCVerbosity = cms.untracked.bool(False),
                         comEnergy = cms.double(13600.),
                         PythiaParameters = cms.PSet(
                                                     pythia8CommonSettingsBlock,
                                                     pythia8CP5SettingsBlock,
                                                     pythia8PSweightsSettingsBlock,
                                                     processParameters = cms.vstring(
                                                     __CHANNEL_DECAY_FRAGMENT__
                                                     ),
                                                     parameterSets = cms.vstring('pythia8CommonSettings',
                                                                                 'pythia8CP5Settings',
                                                                                 'pythia8PSweightsSettings',
                                                                                 'processParameters'
                                                                                 )
                                                     )
                         )


ProductionFilterSequence = cms.Sequence(generator)
"""

In [ ]:
def fragmentsDictCreator(decay_fr):
    dict = { '2024': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
           }
    return dict

In [ ]:
LHEproducer = """
import FWCore.ParameterSet.Config as cms

# link to cards:
# __EXAMPLE__

externalLHEProducer = cms.EDProducer('ExternalLHEProducer',
    args = cms.vstring('__GRIDPACK__'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    generateConcurrently = cms.untracked.bool(False),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
)
"""

In [ ]:
years = ["2024"]
M = ["3000", "4000", "5000", "6000", "7000", "8000", "9000", "10000"]

tot_events = [250000]*len(M)
gridpacks_dict = {}
example_dict = {}
dataset_names = {}

In [ ]:
for year in range(len(years)):
    t_example1 = 'https://gitlab.cern.ch/cms-gen/genproductions_cards/-/tree/master/MadGraph5_aMCatNLO/production/13p6TeV/diquark_Suu_ut'
    t_datasetname_year = 'SuutoTU-TtoBLNu_Par-MSuu-{mass}_TuneCP5_13p6TeV_madgraph-pythia8'
    t_gp_year = "/cvmfs/cms-griddata.cern.ch/phys_generator/gridpacks_tarball/pp/13p6TeV/madgraph/SuuToTU/diquark_Suu_ut_Msuu_{mass}_Run3_el8_amd64_gcc10_CMSSW_12_4_8_tarball.tar.xz"    
    tmp_dataset_dict={}
    tmp_gridpack_dict={}
    tmp_example_dict={}
    dataset_names_year = []
    gp_tprime_year = "gp_tprime_" + years[year]
    gp_tprime_year = []
    
    dataset_names_year = []
    gp_tprime_year = []

    for msuu in M:
        d = {"mass": msuu}
        tmp_example_dict = t_example1
        gp_tprime_year.append(t_gp_year.format_map(d))
        dataset_names_year.append(t_datasetname_year.format_map(d))
        tmp_dataset_dict = dataset_names_year
        tmp_gridpack_dict = gp_tprime_year
        
    gridpacks_dict[years[year]] = tmp_gridpack_dict
    dataset_names[years[year]] = tmp_dataset_dict
    example_dict[years[year]] = tmp_example_dict

## .csv Production

In [ ]:
tau_bug = """
                                                        TauDecays:externalMode=2
"""

In [ ]:
#MadgraphVersion = re.compile("V5_2\.[0-9]\.[0-9]")
process_pythia_map = {}
tmp_process_pythia_map = {}
tmp_process_pythia_map = fragmentsDictCreator(tau_bug)
process_pythia_map.update(tmp_process_pythia_map)

In [ ]:
#for year in range(len(years)):
for year in gridpacks_dict:
    #print(year)
    with open('SuutoTU' + '_' + year +'.csv', 'w') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',',
                        quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(['prepid', 'Dataset name','Events', 'fragment','notes','Generator','mcdbid','time','size'])
        #if year == "2023":
        #    pr = 6027
        #if year == "2023BPix":
        #    pr = 6232
        pr=703    
        k=0
        for i in range(len(M)):
            tmp_fragment=""
            #print(gridpacks_dict[year][i])
            #print(dataset_names[year][i])
            #version = MadgraphVersion.search(path).group(0)
            version="2.9.18"
            dataset_name = dataset_names[year][k]
            events = tot_events[i]
            prepid = 'RunIII2024Summer24wmLHEGS-00' + str(pr)
                
            tmp_fragment = LHEproducer.replace('__GRIDPACK__',gridpacks_dict[year][k]) + '\n' + process_pythia_map[year]
            final_fragment = tmp_fragment.replace('__EXAMPLE__',example_dict[year])
            note = dataset_name.replace('_',' ')
            generators="Madgraph_" + version + "  Pythia8"
            mcdb_id = '0'
            time = '0.01'
            size = '30'
            csvwriter.writerow([prepid, dataset_name, events, final_fragment, note, generators, mcdb_id, time, size])
            pr = pr+1   
            k=k+1